### Build a network model between significant TFs and pathways in cluster 0, one of dendrite cells

**Note:** *The code below may utilize some pre-generated results, such as correlation results between TFs and pathways in a csv file, as well as AUCell-based pathway and TF activities stored in the h5ad file. The code for generating these results can be found in sc_regulatory_network_construction.py. Examples on how to generate these results will be provided here soon. Currently, the code in this notebook focuses on building the gene expression regulatory network for DC Cluster 0.*

In [ ]:
# Make cell wider
# https://stackoverflow.com/questions/21971449/how-do-i-increase-the-cell-width-of-the-jupyter-ipython
# -notebook-in-my-browser
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
# Build a network for the TFs and Pathways
%run ./sc_regulatory_network_construction.py
dir_name = '/Volumes/ssd/AML_P01/evan_scrna_seq/'
file_name = dir_name + 'cor_result_df_cluster_0_061522.csv'
cor_result_df = pd.read_csv(file_name, sep=',')
# Need to reset index
cor_result_df.set_index(keys=cor_result_df.columns[0], drop=True, inplace=True)
print(cor_result_df.shape)

In [ ]:
# Load the data. This is a long step and try to avoid running it!!!
file_name = dir_name + 'CEL211011EL_GEX_final_clustered_annotated_pathways_tfs.h5ad'
adata = sc.read_h5ad(file_name)

In [ ]:
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12,8)
sc.pl.umap(adata, color=['leiden', 'cell.ident'], wspace=0.05, legend_loc='on data', legend_fontsize='large')
# Draw replicate ID in other
# Generate a new column
adata.obs['sample.type'] = adata.obs['replicate.ID'].map(lambda x : x.split('_')[0])
sc.pl.umap(adata, color=['replicate.ID', 'sample.type'], wspace=0.2, legend_fontsize='large')

In [ ]:
# Check AML and WT cell numbers in each cluster
obs_df = pd.DataFrame({'leiden': adata.obs['leiden'],
                       'sample.type': adata.obs['sample.type'],
                       'replicate.ID': adata.obs['replicate.ID']})
HTML(obs_df.groupby(['leiden', 'sample.type']).count().to_html())

In [ ]:
# Focus on cluster 0 and plot violins for some TFs and Pathways in the above correlation DataFrame
adata_cluster_0 = adata[adata.obs['leiden'] == '0']

total_features = 16
# Get tfs
import random
tfs_in_df = list(cor_result_df.index)
tfs_in_df.pop()
tfs_in_df = random.sample(tfs_in_df, total_features)
tfs_in_df.sort()

def create_violin_df(value_key, features, var_name):
    df_cluster_0 = adata_cluster_0.obsm[value_key]
    df_cluster_0 = df_cluster_0[features]
    df_cluster_0['sample.type'] = adata_cluster_0.obs['sample.type']
    df_cluster_0 = pd.melt(df_cluster_0, 
                           value_vars=df_cluster_0.columns,
                           id_vars='sample.type',
                           var_name=var_name,
                           value_name='AUCell')
    return df_cluster_0

# Create a TF for plot
tfs_aucell_df_cluster_0 = create_violin_df(pa.TF_AUCELL_KEY, tfs_in_df, 'TF')

# Get pathways
pathways = {p[0:p.rindex('_')] for p in cor_result_df.columns}
# Convert to list
pathways_in_df = list(pathways)
pathways_in_df = random.sample(pathways_in_df, total_features)
pathways_in_df.sort()
# Create DF
pathways_aucell_df_cluster_0 = create_violin_df(pa.AUCELL_KEY, pathways_in_df, 'Pathway')

def violin_plot_tfs_pathways(tfs_aucell_df_cluster_0,
                             pathways_aucell_df_cluster_0):
    # Plot with two rows
    fig, axs = plt.subplots(ncols=1, nrows=2)
    fig.set_figwidth(30)
    fig.set_figheight(12)
    sns.violinplot(x=tfs_aucell_df_cluster_0['TF'],
                   y=tfs_aucell_df_cluster_0['AUCell'],
                   hue=tfs_aucell_df_cluster_0['sample.type'],
                   split=True,
                   ax=axs[0])
    g = sns.violinplot(x=pathways_aucell_df_cluster_0['Pathway'],
                   y=pathways_aucell_df_cluster_0['AUCell'],
                   hue=pathways_aucell_df_cluster_0['sample.type'],
                   split=True,
                   ax=axs[1])
    g.set_xticklabels(g.get_xticklabels(), rotation=90)

violin_plot_tfs_pathways(tfs_aucell_df_cluster_0, pathways_aucell_df_cluster_0)
#TODO: May need to adjust the threshol fdr used to select TFs and pathways. Probably also need to consider the actual stats (e.g. folder difference, etc.)

In [ ]:
param_cutoff = 0.5
param_cutoff_str = '0_5'
date = '072622'

network = build_network_for_tfs_pathways(cor_result_df, param_cutoff=param_cutoff)
# Cannot work with absolute file path!!!
file_name = 'network' + '_' + param_cutoff_str + '_' + date + '.html'
from IPython.core.display import display, HTML
display(HTML("<a href=\"file:{}\">Click to see the network</a>".format(os.path.abspath(file_name))))
# Cannot put the following code in the external script. Otherwise, it cannot show the network.
# Also it cannot work in a function!!!
network_display_width = 800
network_display_height = 800
ng = display_network(network, width=network_display_width, height=network_display_height)
ng.force_atlas_2based()
ng.show(file_name)

In [ ]:
pathway_gmt_file = '../resources/MouseReactomePathways_Rel_79_122921.gmt'
pathway2genes = pa._load_reactome_gmt(pathway_gmt_file)
tf_file = '../resources/dorothea_mm.tsv'
# For the overlap analysis, we pick up as many targets as possible
tf2genes = pa.load_dorothea_data(tf_file, ['A', 'B', 'C', 'D', 'E'])
s_no_parent_network, overlap_p_values = simplify_network_for_tfs_pathways(pathway2genes, tf2genes, network, 
                                                                need_parent_pathway=False,
                                                                use_direct_interaction=True,
                                                                p_value_for_direction_interaction=0.01,
                                                                for_pathway_only=False, 
                                                                add_tf_links=True,
                                                                check_with_tf_cor=True,
                                                                adata=adata_cluster_0,
                                                                tf_cor_cutoff=0.25,
                                                                add_pathway_to_tf_links=True,
                                                                delete_pathway_only_component=True,
                                                                file_output=None, 
                                                                total_genes=adata.shape[1])
# Somehow need to show this dataframe in other cell. Otherwise, the above two cannot be displayed at the same time.
HTML(overlap_p_values.sort_values(by = ['Pathway']).to_html())

In [ ]:
overlap_p_values.groupby('TF').min()

In [ ]:
# Display the network
file_name = 's_no_parent_network' + '_' + param_cutoff_str + '_' + date + '.html'
display(HTML("<a href=\"file:{}\">Click to see the network</a>".format(file_name)))
ng = display_network(s_no_parent_network, width=network_display_width * 2, height=network_display_height * 1.5)
ng.force_atlas_2based()
ng.show(file_name)

In [ ]:
# Just want to take a look at the TF network
tfs_in_network = [t for t, a in s_no_parent_network.nodes(data=True) if a['type'] == 'TF']
tf_network = build_tfs_network(tfs_in_network, tf2genes, check_with_tf_cor=True, tf_cor_cutoff=0.25, adata=adata_cluster_0)
file_name = 'tf_network' + '_' + param_cutoff_str + '_' + date + '.html'
display(HTML("<a href=\"file:{}\">Click to see the network</a>".format(file_name)))
ng = display_network(tf_network, width=network_display_width, height=network_display_height)
ng.force_atlas_2based()
ng.show(file_name)

In [ ]:
#%run ./AMLTet2Analysis.py
# With parents
s_network, overlap_p_values = simplify_network_for_tfs_pathways(pathway2genes, tf2genes, network, 
                                                                need_parent_pathway=True,
                                                                use_direct_interaction=True,
                                                                p_value_for_direction_interaction=0.01,
                                                                for_pathway_only=False, 
                                                                add_tf_links=True,
                                                                check_with_tf_cor=True,
                                                                adata=adata_cluster_0,
                                                                tf_cor_cutoff=0.25,
                                                                add_pathway_to_tf_links=True,
                                                                delete_pathway_only_component=True,
                                                                delete_leaf_pathway=True,
                                                                file_output=None, 
                                                                total_genes=adata.shape[1])
file_name = 's_network' + '_' + param_cutoff_str + '_' + date + '.html'
display(HTML("<a href=\"file:{}\">Click to see the network</a>".format(file_name)))
ng = display_network(s_network, width=network_display_width * 2, height=network_display_height * 1.5)
ng.force_atlas_2based()
ng.show(file_name)

In [ ]:
s_leaf_network, overlap_p_values = simplify_network_for_tfs_pathways(pathway2genes, tf2genes, network, 
                                                                need_parent_pathway=True,
                                                                use_direct_interaction=True,
                                                                p_value_for_direction_interaction=0.01,
                                                                for_pathway_only=False, 
                                                                add_tf_links=True,
                                                                check_with_tf_cor=True,
                                                                adata=adata_cluster_0,
                                                                tf_cor_cutoff=0.25,
                                                                add_pathway_to_tf_links=True,
                                                                delete_pathway_only_component=True,
                                                                delete_leaf_pathway=False,
                                                                file_output=None, 
                                                                total_genes=adata.shape[1])
file_name = 's_leaf_network' + '_' + param_cutoff_str + '_' + date + '.html'
display(HTML("<a href=\"file:{}\">Click to see the network</a>".format(file_name)))
ng = display_network(s_leaf_network, width=network_display_width * 2, height=network_display_height * 1.5)
ng.force_atlas_2based()
ng.show(file_name)

In [ ]:
# Try to see TFs and some of pathways activityes in a network. Use s_no_parent_network since it is the simpliest one.
tfs_in_network = [t for t, a in s_no_parent_network.nodes(data=True) if a['type'] == 'TF']
print(len(tfs_in_network))
total_features = total_features if total_features < len(tfs_in_network) else len(tfs_in_network)
# tfs_in_network = random.sample(tfs_in_network, total_features)
tfs_in_network.sort()

pathways_in_network = [t for t, a in s_no_parent_network.nodes(data=True) if a['type'] == 'Pathway']
print(len(pathways_in_network))
pathways_in_network = random.sample(pathways_in_network, len(tfs_in_network))
pathways_in_network.sort()

tfs_aucell_df_network = create_violin_df(pa.TF_AUCELL_KEY, tfs_in_network, 'TF')
pathways_aucell_df_network = create_violin_df(pa.AUCELL_KEY, pathways_in_network, 'Pathway')

violin_plot_tfs_pathways(tfs_aucell_df_network, pathways_aucell_df_network)

In [ ]:
# Let's just look at pathways with IL in their names
il_pathways_in_network = [t for t, a in s_no_parent_network.nodes(data=True) if a['type'] == 'Pathway' and 'Interleukin' in t]
print('Total pathways having IL in their names: {}.'.format(len(il_pathways_in_network)))
il_pathays_df = create_violin_df(pa.AUCELL_KEY, il_pathways_in_network, 'Pathway')
fig, axs = plt.subplots(ncols=1, nrows=1)
fig.set_figwidth(30)
fig.set_figheight(8)
g = sns.violinplot(x=il_pathays_df['Pathway'],
                   y=il_pathays_df['AUCell'],
                   hue=il_pathays_df['sample.type'],
                   split=True, ax=axs)
g.set_xticklabels(g.get_xticklabels(), rotation=90)

### Revisit pathway and TF activity differential analysis in cluster 0 to better understanding the up/down pathways between AML and WT

In [ ]:
adata_cluster_0.obs_vector('sample.type').unique()

#### Pathway activity differential analysis in cluster 0

In [ ]:
pathway_diff_df_cluster_0 = cross_sample_analysis(adata_cluster_0, 
                                                  'AML', 
                                                  'WT', 
                                                  sample_key='sample.type',
                                                  need_diff=True,
                                                  score_type=pa.AUCELL_KEY)
file_name = dir_name + 'pathway_diff_df_cluster_0_' + date + '.csv'
pathway_diff_df_cluster_0.to_csv(file_name)
HTML(pathway_diff_df_cluster_0.to_html())

In [ ]:
%run ./AMLTet2Analysis.py
plot_df_1 = plot_pathways(pathway_diff_df_cluster_0)

In [ ]:
# Let's look at what up what down in AML
_ = plot_pathways(pathway_diff_df_cluster_0, feature_col='median_diff')

<b>Note</b>: As we see in the violin plots, there are multiple modes in the distributions of pathway activities. Therefore a simple median difference may not be sufficient to show the up/down patterns for many pathways. This is served as a quick exploration to see the complex pattern in DC.

#### TF activity differential analysis

In [ ]:
tf_diff_df_cluster_0 = cross_sample_analysis(adata_cluster_0, 
                                                  'AML', 
                                                  'WT', 
                                                  sample_key='sample.type',
                                                  need_diff=True,
                                                  score_type=pa.TF_AUCELL_KEY)
print("Total TF: {}.".format(tf_diff_df_cluster_0.shape[0]))
file_name = dir_name + 'tf_diff_df_cluster_0_' + date + '.csv'
tf_diff_df_cluster_0.to_csv(file_name)
HTML(tf_diff_df_cluster_0.to_html())

In [ ]:
%run ./AMLTet2Analysis.py
tf_diff_df_cluster_0.sort_values(by=['feature'], inplace=True)
_ = plot_tfs(tf_diff_df_cluster_0)

In [ ]:
_ = plot_tfs(tf_diff_df_cluster_0,  'median_diff')

In [ ]:
add_median_to_nodes(s_no_parent_network, tf_diff_df_cluster_0)
add_median_to_nodes(s_network, tf_diff_df_cluster_0)
add_median_to_nodes(s_leaf_network, tf_diff_df_cluster_0)

add_median_to_nodes(s_no_parent_network, pathway_diff_df_cluster_0)
add_median_to_nodes(s_network, pathway_diff_df_cluster_0)
add_median_to_nodes(s_leaf_network, pathway_diff_df_cluster_0)

# Dump the networkx project into graphml for Cytoscape
nx.write_graphml(s_leaf_network, dir_name + 's_leaf_network' + '_' + param_cutoff_str + '_' + date + '.graphml')
nx.write_graphml(s_network, dir_name + 's_network' + '_' + param_cutoff_str + '_' + date + '.graphml')
nx.write_graphml(s_no_parent_network, dir_name + 's_no_parent_network' + '_' + param_cutoff_str + '_' + date + '.graphml')

In [ ]:
# check some features
tf_features = ['Myc', 'Nfkb1', 'Tcf7l2', 'Stat3', 'Nfic', 'Trp53']
tf_violin_plot_df = create_violin_df(pa.TF_AUCELL_KEY, tf_features, 'TF')
pathway_features = ['Formation of a pool of free 40S subunits', 
                    'L13a-mediated translational silencing of Ceruloplasmin expression',
                    'Interleukin-37 signaling',
                    'Regulation of IFNG signaling',
                    'Interleukin-21 signaling']
pathway_violin_plot_df = create_violin_df(pa.AUCELL_KEY, pathway_features, 'Pathway')
violin_plot_tfs_pathways(tf_violin_plot_df, pathway_violin_plot_df)
# sc.pl.umap(adata, color=['leiden', 'Trp53'])

In [ ]:
# Do some plots for the DRSN poster
tfs = [x for x, y in network.nodes(data=True) if y['type'] == 'TF']
tfs_16 = random.sample(tfs, 16)
tfs_16.sort()
fig, axs = plt.subplots(ncols=1, nrows=4)
fig.set_figwidth(30)
fig.set_figheight(30)
for i in range(0, len(tfs_16), 4):
    ax_index = int(i / 4)
    tfs_plot = tfs_16[i:i+4]
    tf_violin_plot_df = create_violin_df(pa.TF_AUCELL_KEY, tfs_plot, 'TF')
    g = sns.violinplot(x=tf_violin_plot_df['TF'],
                       y=tf_violin_plot_df['AUCell'],
                       hue=tf_violin_plot_df['sample.type'],
                       split=True,
                       ax=axs[ax_index])
    g.set_xlabel(g.get_xlabel(), fontsize=24)
    g.set_ylabel(g.get_ylabel(), fontsize=24)
    g.set_xticklabels(g.get_xticklabels(), fontsize=20)

In [ ]:
tf_df = adata_cluster_0.obsm[pa.TF_AUCELL_KEY]
pathway_df = adata_cluster_0.obsm[pa.AUCELL_KEY]
# tf_df = adata.obsm[pa.TF_AUCELL_KEY]
tf1 = 'E2f4'
tf2 = 'Bach1'
fig, axs = plt.subplots(ncols=2, nrows=1)
fig.set_figwidth(30)
fig.set_figheight(8)
g = sns.scatterplot(tf_df[tf1], tf_df[tf2], ax = axs[0])
print(stats.spearmanr(tf_df[tf1], tf_df[tf2]))
g = sns.scatterplot(tf_df['Tcf7l2'], pathway_df['PTK6 Activates STAT3'], ax = axs[1])
# print(tf_df['Tcf7l2'], pathway_df['PTK6 Activates STAT3'])